# Example of DOV search methods for interpretations (geotechnische codering)

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/DOV-Vlaanderen/pydov/master?filepath=docs%2Fnotebooks%2Fsearch_geotechnische_codering.ipynb)

## Use cases explained below
* Get 'geotechnische codering' in a bounding box
* Get 'geotechnische codering' with specific properties within a distance from a point
* Get 'geotechnische codering' in a bounding box with specific properties
* Get 'geotechnische codering' based on fields not available in the standard output dataframe
* Get 'geotechnische codering' data, returning fields not available in the standard output dataframe

In [1]:
%matplotlib inline
import inspect, sys

In [2]:
# check pydov path
import pydov

## Get information about the datatype 'Geotecnische codering'

In [3]:
from pydov.search.interpretaties import GeotechnischeCoderingSearch
itp = GeotechnischeCoderingSearch()

A description is provided for the 'Geotechnische codering' datatype:

In [4]:
itp.get_description()

'Een geotechnische codering van een boring is een codering opgesteld vanuit geotechnisch oogpunt, rekening houdend met informatie uit de lithologie, laboproeven en bijhorende sondering(en).'

The different fields that are available for objects of the 'Geotechnische codering' datatype can be requested with the get_fields() method:

In [5]:
fields = itp.get_fields()

# print available fields
for f in fields.values():
    print(f['name'])

pkey_interpretatie
Type_proef
Proefnummer
pkey_boring
x
y
start_interpretatie_mtaw
diepte_tot_m
gemeente
Auteurs
Datum
Opdrachten
betrouwbaarheid_interpretatie
Geldig_van
Geldig_tot
diepte_laag_van
diepte_laag_tot
hoofdnaam1_grondsoort
hoofdnaam2_grondsoort
bijmenging1_plaatselijk
bijmenging1_hoeveelheid
bijmenging1_grondsoort
bijmenging2_plaatselijk
bijmenging2_hoeveelheid
bijmenging2_grondsoort
bijmenging3_plaatselijk
bijmenging3_hoeveelheid
bijmenging3_grondsoort


You can get more information of a field by requesting it from the fields dictionary:
* *name*: name of the field
* *definition*: definition of this field
* *cost*: currently this is either 1 or 10, depending on the datasource of the field. It is an indication of the expected time it will take to retrieve this field in the output dataframe.
* *notnull*: whether the field is mandatory or not
* *type*: datatype of the values of this field

In [6]:
fields['Datum']

{'name': 'Datum',
 'definition': 'De datum waarop de geotechnische codering werd beschreven.',
 'type': 'date',
 'notnull': False,
 'query': True,
 'cost': 1}

## Example use cases

### Get 'Geotechnische codering' in a bounding box

Get data for all the 'Geotechnische codering' interpretations that are geographically located within the bounds of the specified box.

The coordinates are in the Belgian Lambert72 (EPSG:31370) coordinate system and are given in the order of lower left x, lower left y, upper right x, upper right y.

In [7]:
from pydov.util.location import Within, Box

df = itp.search(location=Within(Box(108281, 197850, 108282, 197851)))
df.head()

[000/001] .


,pkey_interpretatie,pkey_boring,betrouwbaarheid_interpretatie,x,y,start_interpretatie_mtaw,diepte_laag_van,diepte_laag_tot,hoofdnaam1_grondsoort,hoofdnaam2_grondsoort,bijmenging1_plaatselijk,bijmenging1_hoeveelheid,bijmenging1_grondsoort,bijmenging2_plaatselijk,bijmenging2_hoeveelheid,bijmenging2_grondsoort,bijmenging3_plaatselijk,bijmenging3_hoeveelheid,bijmenging3_grondsoort
0,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/2011...,goed,108281.2,197850.2,7.99,0.0,0.5,FZ,NaN,False,N,LE,False,N,PL,NaN,NaN,NaN
1,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/2011...,goed,108281.2,197850.2,7.99,0.5,1.0,LE,NaN,False,V,XZ,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/2011...,goed,108281.2,197850.2,7.99,1.0,3.0,FZ,NaN,False,W,LE,NaN,NaN,NaN,NaN,NaN,NaN
3,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/2011...,goed,108281.2,197850.2,7.99,3.0,4.5,FZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/2011...,goed,108281.2,197850.2,7.99,4.5,5.0,FZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The dataframe contains one 'Geotechnische codering' interpretation where ten layers ('laag') were identified. The available data are flattened to represent unique attributes per row of the dataframe.

Using the *pkey_interpretatie* field one can request the details of this interpretation in a webbrowser:

In [8]:
for pkey_interpretatie in set(df.pkey_interpretatie):
    print(pkey_interpretatie)

https://www.dov.vlaanderen.be/data/interpretatie/2011-172244


### Get 'Geotechnische codering' with specific properties within a distance from a point

Next to querying interpretations based on their geographic location within a bounding box, we can also search for interpretations matching a specific set of properties. For this we can build a query using a combination of the 'Geotechnische codering' fields and operators provided by the WFS protocol.

A list of possible operators can be found below:

In [9]:
[i for i,j in inspect.getmembers(sys.modules['owslib.fes'], inspect.isclass) if 'Property' in i]

['PropertyIsBetween',
 'PropertyIsEqualTo',
 'PropertyIsGreaterThan',
 'PropertyIsGreaterThanOrEqualTo',
 'PropertyIsLessThan',
 'PropertyIsLessThanOrEqualTo',
 'PropertyIsLike',
 'PropertyIsNotEqualTo',
 'PropertyIsNull',
 'SortProperty']

In this example we build a query using the *PropertyIsGreaterThan* and *PropertyIsEqualTo* operators to find all interpretations that are at least 20 m deep, that are deemed appropriate for a range of 1 km from a defined point:

In [10]:
from owslib.fes import And, PropertyIsGreaterThan, PropertyIsEqualTo
from pydov.util.location import WithinDistance, Point

query = And([PropertyIsEqualTo(propertyname='Betrouwbaarheid',
                              literal='goed'),
            PropertyIsGreaterThan(propertyname='diepte_tot_m',
                                 literal='20'),
           ])
            
df = itp.search(query=query, 
                location=WithinDistance(Point(153145, 206930), 1000))

df.head()

[000/003] ...


,pkey_interpretatie,pkey_boring,betrouwbaarheid_interpretatie,x,y,start_interpretatie_mtaw,diepte_laag_van,diepte_laag_tot,hoofdnaam1_grondsoort,hoofdnaam2_grondsoort,bijmenging1_plaatselijk,bijmenging1_hoeveelheid,bijmenging1_grondsoort,bijmenging2_plaatselijk,bijmenging2_hoeveelheid,bijmenging2_grondsoort,bijmenging3_plaatselijk,bijmenging3_hoeveelheid,bijmenging3_grondsoort
0,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1971...,goed,153935.0,207327.0,16.16,0.00,1.75,FZ,LE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1971...,goed,153935.0,207327.0,16.16,1.75,2.25,FZ,LE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1971...,goed,153935.0,207327.0,16.16,2.25,2.70,KL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1971...,goed,153935.0,207327.0,16.16,2.70,3.25,KL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1971...,goed,153935.0,207327.0,16.16,3.25,3.70,KL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Once again we can use the *pkey_interpretatie* as a permanent link to the information of these interpretations:

In [11]:
for pkey_interpretatie in set(df.pkey_interpretatie):
    print(pkey_interpretatie)

https://www.dov.vlaanderen.be/data/interpretatie/2012-180862
https://www.dov.vlaanderen.be/data/interpretatie/2012-180861
https://www.dov.vlaanderen.be/data/interpretatie/2012-180863


### Get 'Geotechnische codering' in a bounding box based on specific properties

We can combine a query on attributes with a query on geographic location to get the interpretations within a bounding box that have specific properties.

The following example requests the interpretations of boreholes only, within the given bounding box.

(Note that the datatype of the *literal* parameter should be a string, regardless of the datatype of this field in the output dataframe.)

In [12]:
from owslib.fes import PropertyIsEqualTo

query = PropertyIsEqualTo(
            propertyname='Type_proef',
            literal='Boring')

df = itp.search(
    location=Within(Box(153145, 206930, 154145, 207930)),
    query=query
    )

df.head()

[000/020] ....cc...c..........


,pkey_interpretatie,pkey_boring,betrouwbaarheid_interpretatie,x,y,start_interpretatie_mtaw,diepte_laag_van,diepte_laag_tot,hoofdnaam1_grondsoort,hoofdnaam2_grondsoort,bijmenging1_plaatselijk,bijmenging1_hoeveelheid,bijmenging1_grondsoort,bijmenging2_plaatselijk,bijmenging2_hoeveelheid,bijmenging2_grondsoort,bijmenging3_plaatselijk,bijmenging3_hoeveelheid,bijmenging3_grondsoort
0,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1970...,goed,153963.0,207151.0,15.54,0.00,0.75,FZ,LE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1970...,goed,153963.0,207151.0,15.54,0.75,1.25,FZ,LE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1970...,goed,153963.0,207151.0,15.54,1.25,1.75,LE,NaN,False,N,FZ,NaN,NaN,NaN,NaN,NaN,NaN
3,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1970...,goed,153963.0,207151.0,15.54,1.75,1.90,KL,NaN,False,V,FZ,False,W,GL,NaN,NaN,NaN
4,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1970...,goed,153963.0,207151.0,15.54,1.90,2.75,KL,NaN,False,V,FZ,False,W,GL,NaN,NaN,NaN


We can look at one of the interpretations in a webbrowser using its *pkey_interpretatie*:

In [13]:
for pkey_interpretatie in set(df.pkey_interpretatie):
    print(pkey_interpretatie)

https://www.dov.vlaanderen.be/data/interpretatie/2013-182275
https://www.dov.vlaanderen.be/data/interpretatie/2013-182282
https://www.dov.vlaanderen.be/data/interpretatie/2012-180861
https://www.dov.vlaanderen.be/data/interpretatie/2012-180855
https://www.dov.vlaanderen.be/data/interpretatie/2012-180863
https://www.dov.vlaanderen.be/data/interpretatie/2013-182278
https://www.dov.vlaanderen.be/data/interpretatie/2013-182280
https://www.dov.vlaanderen.be/data/interpretatie/2012-180853
https://www.dov.vlaanderen.be/data/interpretatie/2012-180851
https://www.dov.vlaanderen.be/data/interpretatie/2012-180852
https://www.dov.vlaanderen.be/data/interpretatie/2013-182276
https://www.dov.vlaanderen.be/data/interpretatie/2013-182359
https://www.dov.vlaanderen.be/data/interpretatie/2013-182358
https://www.dov.vlaanderen.be/data/interpretatie/2012-180866
https://www.dov.vlaanderen.be/data/interpretatie/2013-182279
https://www.dov.vlaanderen.be/data/interpretatie/2013-182281
https://www.dov.vlaander

### Get 'Geotechnische codering' based on fields not available in the standard output dataframe

To keep the output dataframe size acceptable, not all available WFS fields are included in the standard output. However, one can use this information to select interpretations as illustrated below.

For example, make a selection of the interpretations in municipality the of Antwerp, before 1/1/1990:

!*remark: mind that the municipality attribute is merely an attribute that is defined by the person entering the data. It can be ok, empty, outdated or wrong*!

In [14]:
from owslib.fes import And, PropertyIsEqualTo, PropertyIsLessThan

query = And([PropertyIsEqualTo(propertyname='gemeente',
                               literal='Antwerpen'),
             PropertyIsLessThan(propertyname='Datum', 
                                 literal='2010-01-01')]
            )
df = itp.search(query=query,
                return_fields=('pkey_interpretatie', 'Datum'))
df.head()

,pkey_interpretatie,Datum
0,https://www.dov.vlaanderen.be/data/interpretat...,2006-03-24
1,https://www.dov.vlaanderen.be/data/interpretat...,2006-02-17
2,https://www.dov.vlaanderen.be/data/interpretat...,2006-02-27
3,https://www.dov.vlaanderen.be/data/interpretat...,2006-02-17
4,https://www.dov.vlaanderen.be/data/interpretat...,2005-02-15


### Get 'Geotechnische codering' data, returning fields not available in the standard output dataframe

As denoted in the previous example, not all available fields are available in the default output frame to keep its size limited. However, you can request any available field by including it in the *return_fields* parameter of the search:

In [15]:
query = PropertyIsEqualTo(
            propertyname='gemeente',
            literal='Leuven')

df = itp.search(query=query,
                return_fields=('pkey_interpretatie', 'pkey_boring',
                               'x', 'y', 'Z_mTAW', 'gemeente', 'Auteurs', 'Proefnummer'))

df.head()

InvalidFieldError: Unknown return field: 'Z_mTAW'. Did you mean 'start_interpretatie_mtaw'?

## Visualize results

Using Folium, we can display the results of our search on a map.

In [16]:
# import the necessary modules (not included in the requirements of pydov!)
import folium
from folium.plugins import MarkerCluster
from pyproj import Transformer

In [17]:
# convert the coordinates to lat/lon for folium
def convert_latlon(x1, y1):
    transformer = Transformer.from_crs("epsg:31370", "epsg:4326", always_xy=True)
    x2,y2 = transformer.transform(x1, y1)
    return x2, y2

df['lon'], df['lat'] = zip(*map(convert_latlon, df['x'], df['y'])) 
# convert to list
loclist = df[['lat', 'lon']].values.tolist()

In [18]:
# initialize the Folium map on the centre of the selected locations, play with the zoom until ok
fmap = folium.Map(location=[df['lat'].mean(), df['lon'].mean()], zoom_start=12)
marker_cluster = MarkerCluster().add_to(fmap)
for loc in range(0, len(loclist)):
    folium.Marker(loclist[loc], popup=df['Proefnummer'][loc]).add_to(marker_cluster)
fmap
